<a href="https://colab.research.google.com/github/giuliocapecchi/IR_project/blob/main/IR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torch matplotlib nltk tqdm gdown ir_datasets humanize 

Note: you may need to restart the kernel to use updated packages.


# 1. Download and prepare the collection

In [2]:
# chosen_collection can be one of ["vaswani", "MSMARCO"]

chosen_collection = "MSMARCO"

In [3]:
import gdown
import ir_datasets
import pandas as pd
import os

if chosen_collection not in ["vaswani", "MSMARCO"]:
    raise ValueError("chosen_collection must be one of ['vaswani', 'MSMARCO']")

if chosen_collection == "MSMARCO":
    url_collection = 'https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn'
    url_stats = 'https://drive.google.com/uc?id=1ruBLKGkKvfFlPMdjVUGkR09_f-528fEz'
    url_lex = 'https://drive.google.com/uc?id=151vGXMMwslHM-Vv8vKRATNLazyAXDM1A'
    url_inv = 'https://drive.google.com/uc?id=1UUL0SkjFq4V9tNGiabRo27GfH3XsUuyk'
    url_doc = 'https://drive.google.com/uc?id=1p-AChVgbUN4nIzFO55-pm8CtCAghobWC'
    if not os.path.exists('collection.tsv'):
        gdown.download(url_collection, 'collection.tsv', quiet=False)
    
    """os.makedirs('./pickles', exist_ok=True)
    if not os.path.exists('./pickles/stats.pkl'):
        gdown.download(url_stats, './pickles/stats.pkl', quiet=False)
    if not os.path.exists('./pickles/lex.pkl'):
        gdown.download(url_lex, './pickles/lex.pkl', quiet=False)
    if not os.path.exists('./pickles/inv.pkl'):
        gdown.download(url_inv, './pickles/inv.pkl', quiet=False)
    if not os.path.exists('./pickles/doc.pkl'):
        gdown.download(url_doc, './pickles/doc.pkl', quiet=False)"""

    #df = pd.read_csv('collection.tsv', sep='\t', header=None, names=['doc_id', 'text'])

elif chosen_collection == "vaswani":
    vaswani_dataset = ir_datasets.load("vaswani")
    docs = list(vaswani_dataset.docs_iter())
    df = pd.DataFrame(docs)
    df['doc_id'] = (df['doc_id'].astype(int) - 1).astype(str)
    # rimuovi i \n da ogni documento
    df['text'] = df['text'].str.replace('\n', ' ')
    df.to_csv('vaswani.tsv', sep='\t', header=False, index=False)

In [4]:
# #let's not truncate Pandas output too much
# pd.set_option('display.max_colwidth', 50) # mettici 150


# print(df.head(5)) # returns the first N rows

In [5]:
import re
import string
import nltk


nltk.download("stopwords", quiet=True)
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
STEMMER = nltk.stem.PorterStemmer()

def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand and special chars
    s = re.sub(r"[‘’´“”–-]", "'", s.replace("&", " and ")) # this replaces & with 'and' and normalises quotes
    # acronyms
    s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s) # this removes dots that are not part of an acronym
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = re.sub(r"\s+", " ", s).strip()
    # tokenisation
    tokens = [t for t in s.split() if t not in STOPWORDS]
    # stemming
    tokens = [STEMMER.stem(t) for t in tokens]
    return tokens

In [6]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [7]:
import pickle
import humanize
import os

def print_pickled_size(var_name, var):
    # If the 'tmp' directory does not exist, we first create it
    os.makedirs('./tmp', exist_ok=True)
    with open(f"./tmp/{var_name}.pickle", 'wb') as f:
        pickle.dump(var, f)
    print(f'{var_name} requires {humanize.naturalsize(os.path.getsize(f"./tmp/{var_name}.pickle"))}')
    os.remove(f"./tmp/{var_name}.pickle")
    os.removedirs('./tmp')


def vbyte_encode(number):
    bytes_list = bytearray()
    while True:
        byte = number & 0x7F # Prendi i 7 bit meno significativi -> 0111 1111 = 0x7F
        number >>= 7 # Shifta a destra di 7 bit
        if number:
            bytes_list.append(byte) # Aggiungo i 7 bit al risultato
        else:
            bytes_list.append(0x80 | byte) # Aggiungo i 7 bit con il bit di continuazione, 0x80 = 1000 0000
            break
    return bytes(bytes_list)

def vbyte_decode(bytes_seq):
    number = 0
    for i, byte in enumerate(bytes_seq):
        number |= (byte & 0x7F) << (7 * i)
        if byte & 0x80:
            break
    return number

def decode_concatenated_vbyte(encoded_bytes):
    decoded_numbers = []
    current_number = 0
    shift_amount = 0
    
    for byte in encoded_bytes:
        if byte & 0x80:  # Bit di continuazione trovato, fine del numero
            current_number |= (byte & 0x7F) << shift_amount
            decoded_numbers.append(current_number)
            current_number = 0
            shift_amount = 0
        else:  # Continuo a comporre il numero
            current_number |= (byte & 0x7F) << shift_amount
            shift_amount += 7
    
    return decoded_numbers

In [8]:
# from collections import namedtuple


# class MSMarcoDataset:
#     """
#     This class that takes the dataframe we created before with columns 'docno' and 'text', and creates a list of namedtuples
#     """
#     def __init__(self, df):
#         self.docs = [Document(row.doc_id, row.text) for row in df.itertuples()]

#     def docs_iter(self):
#         return iter(self.docs)

#     def docs_count(self):
#         return len(self.docs)

# Document = namedtuple('Document', ['doc_id', 'text']) # must define what a document is


# from collections import Counter
# from tqdm.auto import tqdm

# @profile
# def old_build_index(dataset):
#     lexicon = {}
#     doc_index = []
#     inv_d, inv_f = {}, {}
#     termid = 0

#     num_docs = 0
#     total_dl = 0
#     total_toks = 0
#     for docid, doc in tqdm(enumerate(dataset.docs_iter()), desc='Indexing', total=dataset.docs_count()):
#         tokens = preprocess(doc.text)
#         #print(tokens)
#         token_tf = Counter(tokens)
#         for token, tf in token_tf.items():
#             if token not in lexicon:
#                 lexicon[token] = [termid, 0, 0]
#                 inv_d[termid], inv_f[termid] =  [], []
#                 termid += 1
#             token_id = lexicon[token][0] # prendo il termid
#             inv_d[token_id].append(docid) # aggiungo il docid alla lista dei docid in cui compare il termine
#             inv_f[token_id].append(tf) # aggiungo il tf alla lista dei tf in cui compare il termine
#             lexicon[token][1] += 1 # incremento il df
#             lexicon[token][2] += tf # tf è quanto compare il termine nel documento
#         doclen = len(tokens)
#         doc_index.append((str(doc.doc_id), doclen))
#         total_dl += doclen
#         num_docs += 1

#     # Compress the inv_d and inv_f lists
#     for term, (termid, df, _) in lexicon.items():
#         # Compress the docids
#         encoded_list = [vbyte_encode(x) for x in inv_d[termid]]
#         concatenated_encoded = b''.join(encoded_list)
#         assert inv_d[termid] == decode_concatenated_vbyte(concatenated_encoded), "Compression/Decompression mismatch!"
#         inv_d[termid] = concatenated_encoded
#         # Compress the frequencies
#         encoded_list = [vbyte_encode(x) for x in inv_f[termid]]
#         concatenated_encoded = b''.join(encoded_list)
#         assert inv_f[termid] == decode_concatenated_vbyte(concatenated_encoded), "Compression/Decompression mismatch!"
#         inv_f[termid] = concatenated_encoded
        

    
#     stats = {
#         'num_docs': 1 + docid, # docid starts from 0
#         'num_terms': len(lexicon),
#         'num_tokens': total_dl,
#     }
#     return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats


# dataset = MSMarcoDataset(df)

# expected_lexicon, expected_inv, expected_doc_index, expected_stats = old_build_index(dataset)

First preprocess the documents:

In [9]:
import csv
from collections import Counter
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import string
import nltk


nltk.download("stopwords", quiet=True)
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
STEMMER = nltk.stem.PorterStemmer()

def process_document(docid, text):
    """
    Preprocess a document, then returns the docid, the term frequencies, and the doclen. 
    """
    try:
        # lowercasing, special chars, quotes, ampersand
        text = text.lower()
        text = re.sub(r"[‘’´“”–-]", "'", text.replace("&", " and "))
        text = re.sub(r"\.(?!(\S[^. ])|\d)", "", text)
        text = text.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
        text = re.sub(r"\s+", " ", text).strip()

        # tokenisation, stopword removal, stemming
        tokens = [STEMMER.stem(t) for t in text.split() if t not in STOPWORDS]
        
        # term frequencies and document length
        token_tf = Counter(tokens) 
        token_tf = dict(token_tf) # just to have a serializable object
        doclen = len(tokens) # document length

        return (int(docid), token_tf, doclen)
    except Exception as e:
        print(f"Error processing document {docid}: {e}")
        return None


def process_in_chunks(file_path, batch_size=10000, num_cores=-1):
    """
    Process a TSV file in chunks of batch_size, using num_cores CPU cores (default: all). 
    """
    results = []
    docid_counter = 0  # counter for docids

    # count the total number of lines in the file, just for see tqdm progress bar
    with open(file_path, mode="r", encoding="utf-8") as f:
        total_lines = sum(1 for _ in f) 

    with open(file_path, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")
        batch = []
        for line in tqdm(reader, desc="Processing documents", total=total_lines):
            text = line[1] 
            batch.append((docid_counter, text)) 
            docid_counter += 1
            # when the batch is full, process it
            if len(batch) >= batch_size:
                batch_results = Parallel(n_jobs=num_cores)( 
                    delayed(process_document)(docid, text) for docid, text in batch
                )
                results.extend(batch_results) # append the results to the final list
                batch = []  # reset the batch

        # process the last batch
        if batch:
            batch_results = Parallel(n_jobs=num_cores)(
                delayed(process_document)(docid, text) for docid, text in batch
            )
            results.extend(batch_results)

    return results


if chosen_collection == "vaswani":
    results = process_in_chunks("vaswani.tsv")
elif chosen_collection == "MSMARCO": # requires approx 11 minutes
    results = process_in_chunks("collection.tsv")

Processing documents: 100%|██████████| 8841823/8841823 [10:01<00:00, 14705.30it/s]


In [10]:
# time : approx 5 minutes
import gc

# leggi e metti dentro df 'processed_collection.csv'
df = pd.DataFrame(results, columns=["doc_id", "token_tf", "doclen"])

del results  # Libera la memoria
gc.collect()

print("created the dataframe and freed memory")
df.to_csv("processed_collection.csv", index=False)
del df 
gc.collect()

created the dataframe and freed memory


0

In [12]:
# time : approx 35s
from tqdm import trange
import pandas as pd
import os
import shutil
    
if os.path.exists('./results_parquets'):
    shutil.rmtree('./results_parquets')
os.makedirs('./results_parquets', exist_ok=True)

df = pd.read_csv("processed_collection.csv")
batch_size = 1_000_000 # rows per batch
num_files = (len(df) + batch_size - 1) // batch_size  # number of files to save

for i in trange(num_files, desc="Saving batches"):
    start_row = i * batch_size
    end_row = start_row + batch_size
    df_batch = df.iloc[start_row:end_row]
    
    df_batch.to_parquet(f"./results_parquets/output_batch_{i+1}.parquet", index=False)

del df
gc.collect()

Saving batches: 100%|██████████| 9/9 [00:08<00:00,  1.06it/s]


210

In [13]:
if chosen_collection == "vaswani":
    print(f"Collection size: {humanize.naturalsize(os.path.getsize('vaswani.tsv'))}")
elif chosen_collection == "MSMARCO":
    print(f"Collection size: {humanize.naturalsize(os.path.getsize('collection.tsv'))}")


# print os size of the parquet files inside results_parquets
total_size = 0
for file in os.listdir("./results_parquets"):
    total_size += os.path.getsize(os.path.join("./results_parquets", file))
print(f"Processed collection size: {humanize.naturalsize(total_size)}")

Collection size: 3.1 GB
Processed collection size: 1.2 GB


Then we utilize the preprocessed results (which will be more manageable in memory) to build the final structures:


In [ ]:
import os
import pandas as pd
from collections import defaultdict
from joblib import Parallel, delayed
from tqdm import tqdm
import ast

# break the list into chunks of size chunk_size
def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

# elaborate a single parquet file
def process_parquet(parquet_file):
    lexicon = {}
    doc_index = []
    inv_d = defaultdict(list)
    inv_f = defaultdict(list)
    termid = 0
    num_docs = 0
    total_dl = 0

    df = pd.read_parquet(parquet_file)

    for result in df.itertuples(index=False, name=None):
        docid, token_tf, doclen = result
        if isinstance(token_tf, str):
            try:
                token_tf = ast.literal_eval(token_tf) # convert the string back to a dictionary
                #print("Parsed token_tf", token_tf)
            except Exception as e:
                print(f"Error parsing token_tf for document {docid}: {e}")
                continue
            

        for token, tf in token_tf.items():
            if token not in lexicon:
                lexicon[token] = [termid, 0, 0]  # termid, df, tf
                termid += 1

            token_id = lexicon[token][0]
            inv_d[token_id].append(docid)
            inv_f[token_id].append(tf)
            lexicon[token][1] += 1  # increase document frequency (df)
            lexicon[token][2] += tf  # increase term frequency (tf)

        # finally add the document to the doc_index
        doc_index.append((str(docid), doclen))
        total_dl += doclen
        num_docs += 1

    return lexicon, inv_d, inv_f, doc_index, num_docs, total_dl



# combine the results from all the parquet files
def combine_results(results):
    lexicon = {}
    inv_d = defaultdict(list)
    inv_f = defaultdict(list)
    doc_index = []
    num_docs = 0
    total_dl = 0

    for lex, inv_d_part, inv_f_part, doc_index_part, part_num_docs, part_total_dl in results:
        lexicon.update(lex) # with update we merge the dictionaries
        for termid, docids in inv_d_part.items():
            inv_d[termid].extend(docids)
        for termid, freqs in inv_f_part.items():
            inv_f[termid].extend(freqs)
        doc_index.extend(doc_index_part)
        num_docs += part_num_docs
        total_dl += part_total_dl


    # compress the inv_d and inv_f lists
    for term, (termid, df, _) in tqdm(lexicon.items(), desc="Compressing"):
        # compress the docids
        encoded_list = [vbyte_encode(x) for x in inv_d[termid]]
        concatenated_encoded = b''.join(encoded_list)
        # assert inv_d[termid] == decode_concatenated_vbyte(concatenated_encoded), "Compression/Decompression mismatch!"
        inv_d[termid] = concatenated_encoded
        # compress the frequencies
        encoded_list = [vbyte_encode(x) for x in inv_f[termid]]
        concatenated_encoded = b''.join(encoded_list)
        #assert inv_f[termid] == decode_concatenated_vbyte(concatenated_encoded), "Compression/Decompression mismatch!"
        inv_f[termid] = concatenated_encoded

    return lexicon, {'docids': dict(inv_d), 'freqs': dict(inv_f)}, doc_index, num_docs, total_dl


def build_index_from_parquet_files(parquet_dir, batch_size=2):
    """
    lex,inv,doc_index,stats from a directory containing Parquet files.
    """
    # first we get the list of Parquet files from the directory
    parquet_files = [os.path.join(parquet_dir, f) for f in os.listdir(parquet_dir) if f.endswith('.parquet')]

    # split the list of files into batches (by default 2 files per batch)
    file_batches = list(chunk_list(parquet_files, batch_size))

    # process each batch in parallel
    all_results = []
    for batch in tqdm(file_batches, desc="Processing file batches"):
        results = list(
            Parallel(n_jobs=-1, backend='loky')(delayed(process_parquet)(parquet_file) for parquet_file in batch)
        )
        all_results.extend(results)

    # finally combine all the results
    lexicon, inv, doc_index, num_docs, total_dl = combine_results(all_results)

    stats = {
        'num_docs': num_docs,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }

    return lexicon, inv, doc_index, stats

Compressing: 100%|██████████| 1793595/1793595 [08:37<00:00, 3465.21it/s] 


In [15]:
import math

class InvertedIndex:

    class PostingListIterator:
        def __init__(self, docids, freqs, doc):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.doc = doc

        def docid(self):
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]

        def score(self):
            if self.is_end_list():
                return math.inf
            return self.freqs[self.pos]/self.doc[self.docid()][1]

        def next(self, target = None):
            if not target:
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():
                    try:
                        self.pos = self.docids.index(target, self.pos)
                    except ValueError:
                        self.pos = len(self.docids)

        def is_end_list(self):
            return self.pos == len(self.docids)


        def len(self):
            return len(self.docids)


    def __init__(self, lex, inv, doc, stats):
        self.lexicon = lex
        self.inv = inv
        self.doc = doc
        self.stats = stats

    def num_docs(self):
        return self.stats['num_docs']

    def get_posting(self, termid):
        # Extract the encoded docids and freqs
        docids_encoded = self.inv['docids'][termid]
        freqs_encoded = self.inv['freqs'][termid]
        # Decode the docids and freqs before returning the iterator
        docids = decode_concatenated_vbyte(docids_encoded)
        freqs = decode_concatenated_vbyte(freqs_encoded)
    
        return InvertedIndex.PostingListIterator(docids, freqs, self.doc)
    

    def get_termids(self, tokens):
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        return [self.get_posting(termid) for termid in termids]

Now build up the index for the chosen collection. It is built only if a pickled version of its components doesn't exist already :

In [26]:
import pickle
import os

# If the 'pickles' directory does not exist, we first create it
os.makedirs('./pickles', exist_ok=True)

try: # try to open the pickled files, else build the index
    with open('./pickles/lex.pkl', 'rb') as f:
        lex = pickle.load(f)
    with open('./pickles/inv.pkl', 'rb') as f:
        inv = pickle.load(f)
    with open('./pickles/doc.pkl', 'rb') as f:
        doc = pickle.load(f)
    with open('./pickles/stats.pkl', 'rb') as f:
        stats = pickle.load(f)
    print("Index loaded from pickles")

except:
    lex, inv, doc, stats = build_index_from_parquet_files("results_parquets", batch_size=2)
    print("lex, inv, doc, and stats built, now pickling them")

    # pickle lex, inv, doc, stats
    with open('./pickles/lex.pkl', 'wb') as f:
        pickle.dump(lex, f)

    with open('./pickles/inv.pkl','wb') as f:
        pickle.dump(inv, f)

    with open('./pickles/doc.pkl', 'wb') as f:
        pickle.dump(doc, f)

    with open('./pickles/stats.pkl', 'wb') as f:
        pickle.dump(stats, f)

    print("Index built and pickled")



print(f"Numero di documenti: {stats['num_docs']}")
print(f"Numero di termini nel lessico: {stats['num_terms']}")
print(f"Totale token: {stats['num_tokens']}")

Index loaded from pickles
Numero di documenti: 8841823
Numero di termini nel lessico: 1793595
Totale token: 306696510


In [27]:
if chosen_collection == "vaswani":
    # delete the pickles folder
    import shutil
    shutil.rmtree('./pickles')

In [28]:
# try:
#     assert lex == expected_lexicon, "Lexicon does not match expected"
    
#     # Ordinare le liste prima di confrontarle
#     # assert sorted(inv['docids']) == sorted(expected_inv['docids']), "Inverted document index does not match expected"
#     # assert sorted(inv['freqs']) == sorted(expected_inv['freqs']), "Inverted frequencies do not match expected"
    
#     assert inv['docids'] == expected_inv['docids'], "Inverted document index does not match expected"
#     assert inv['freqs'] == expected_inv['freqs'], "Inverted frequencies do not match expected"
    
    
    
#     # Ordinare l'indice dei documenti se è una lista
#     assert doc == expected_doc_index, "Document index does not match expected"
    
#     # Confronto per le statistiche, se sono dizionari o strutture simili
#     assert stats == expected_stats, "Stats do not match expected"
    
# except AssertionError as e:
#     print(e)

In [29]:
def print_index(lex, inv, doc, stats): 
    print("**********************************************************************************\nFirst 5 elements of lexicon:")
    print(list(lex.items())[:5])

    print("\nFirst 5 elements of inv:\n\tFirst 5 elements of inverted index docids:")
    print("\t",{k: decode_concatenated_vbyte(v)[:5] for k, v in sorted(list(inv['docids'].items()))[:5]})
    print("\tFirst 5 elements of inverted index freqs:")
    print("\t",{k: decode_concatenated_vbyte(v)[:5] for k, v in list(inv['freqs'].items())[:5]})

    print("\nFirst 5 elements of document index:")
    print(doc[:5])

    print("\nStats:")
    print(stats,"\n**********************************************************************************")

print_index(lex, inv, doc, stats)
#print_index(expected_lexicon, expected_inv, expected_doc_index, expected_stats)

**********************************************************************************
First 5 elements of lexicon:
[('presenc', [3779, 2700, 2950]), ('commun', [2596, 14595, 18302]), ('amid', [36147, 254, 273]), ('scientif', [3013, 2508, 2893]), ('mind', [252, 5448, 6117])]

First 5 elements of inv:
	First 5 elements of inverted index docids:
	 {0: [0, 61, 203, 235, 323], 1: [0, 10, 11, 12, 13], 2: [128, 110, 129, 40, 169], 3: [0, 434, 461, 699, 1541], 4: [0, 41, 183, 184, 604]}
	First 5 elements of inverted index freqs:
	 {0: [1, 1, 2, 1, 1], 1: [1, 1, 2, 2, 3], 2: [129, 129, 129, 129, 129], 3: [2, 1, 1, 1, 1], 4: [1, 1, 1, 1, 1]}

First 5 elements of document index:
[('0', 27), ('1', 19), ('2', 24), ('3', 30), ('4', 31)]

Stats:
{'num_docs': 8841823, 'num_terms': 1793595, 'num_tokens': 306696510} 
**********************************************************************************


We have all the elements necessary to build the Inverted Index

In [30]:
inv_index = InvertedIndex(lex, inv, doc, stats)

# save the InvertedIndex object to a pickle file
with open('./pickles/inv_index.pkl', 'wb') as f:
    pickle.dump(inv_index, f)

# print the size of the inv_index pickle file
with open('./pickles/inv_index.pkl', 'rb') as f:
    inv_index_size = humanize.naturalsize(os.path.getsize(f"./pickles/inv_index.pkl"))
    print(f"inv_index requires {inv_index_size}")

inv_index requires 1.6 GB


In [31]:
# old_inv_index = InvertedIndex(expected_lexicon,expected_inv,expected_doc_index,expected_stats)
# print_pickled_size('old_inv_index',old_inv_index)


# assert inv_index.num_docs() == old_inv_index.num_docs(), "Not equal"
# assert inv_index.get_termids(["the", "cat"]) == old_inv_index.get_termids(["the", "cat"]), "Not equal"

# postings = zip(inv_index.get_postings([0, 1, 100]), old_inv_index.get_postings([0, 1,100]))
# for posting, old_posting in postings:
#     while not posting.is_end_list():
#         assert posting.docid() == old_posting.docid(), "Not equal"
#         assert posting.score() == old_posting.score(), "Not equal"
#         posting.next() # TODO sEI QUI , adavance?
#         old_posting.next()

---

# 2. Download and prepare queries

Aggiungiamo anche le query per il dataset scelto

In [ ]:
if chosen_collection == "MSMARCO":
    if not os.path.exists('queries.tar.gz'):
        url = 'https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz'
        gdown.download(url, 'queries.tar.gz', quiet=False)
        !tar -xzf queries.tar.gz
    queries = pd.read_csv('queries.eval.tsv', sep='\t', header=None)
    print("Number of queries: ",len(queries))
elif chosen_collection == "vaswani":
    # Converte le query in un DataFrame
    queries = pd.DataFrame(vaswani_dataset.queries_iter())
    queries.columns = ['qid', 'query']
    print(queries.head(2))
    print("Number of queries: ",len(list(vaswani_dataset.queries_iter()))) 

In [ ]:
print(queries.head(5))

In [ ]:
from collections import namedtuple


class MSMarcoQueries:
    def __init__(self, df):
        self.queries = [Query(row.query_id, row.text) for row in df.itertuples()]

    def queries_iter(self):
        return iter(self.queries)

    def queries_count(self):
        return len(self.queries)


Query = namedtuple('Query', ['query_id', 'text'])


queries.columns = ['query_id', 'text']
queries_dataset = MSMarcoQueries(queries)
print("The number of queries is: ", queries_dataset.queries_count())

Let's prepare the functions necessary to perform TAAT and DAAT query processing

First, we need a TopQueue class, which stores the top  K  (score, docid) tuples, using an heap 

In [ ]:
import heapq

class TopQueue:
    def __init__(self, k=10, threshold=0.0):
        self.queue = []
        self.k = k
        self.threshold = threshold

    def size(self):
        return len(self.queue)

    def would_enter(self, score):
        return score > self.threshold

    def clear(self, new_threshold=None):
        self.queue = []
        if new_threshold:
            self.threshold = new_threshold

    def __repr__(self):
        return f'<{self.size()} items, th={self.threshold} {self.queue}'

    def insert(self, docid, score):
        if score > self.threshold:
            if self.size() >= self.k:
                heapq.heapreplace(self.queue, (score, docid))
            else:
                heapq.heappush(self.queue, (score, docid))
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True
        return False

### TAAT

In [ ]:
from collections import defaultdict

def taat(postings, k=10):
    A = defaultdict(float)
    for posting in postings:
        current_docid = posting.docid()
        while current_docid != math.inf:
            A[current_docid] += posting.score()
            posting.next()
            current_docid = posting.docid()
    top = TopQueue(k)
    for docid, score in A.items():
        top.insert(docid, score)
    return sorted(top.queue, reverse=True)

def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return taat(postings)

### DAAT

In [ ]:
import math

def min_docid(postings):
    min_docid = math.inf
    for p in postings:
        if not p.is_end_list():
            min_docid = min(p.docid(), min_docid)
    return min_docid

def daat(postings, k=10):
    top = TopQueue(k)
    current_docid = min_docid(postings)
    while current_docid != math.inf:
        score = 0
        next_docid = math.inf
        for posting in postings:
            if posting.docid() == current_docid:
                score += posting.score()
                posting.next()
            if not posting.is_end_list():
                next_docid = posting.docid()
        top.insert(current_docid, score)
        current_docid = next_docid
    return sorted(top.queue, reverse=True)

def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return daat(postings)

In [ ]:
@profile
def query_processing(queries_iter, fn):
    for q in queries_iter:
        query = preprocess(q.text)
        termids = inv_index.get_termids(query)
        postings = inv_index.get_postings(termids)
        res = fn(postings)

In [ ]:
query_processing(queries_dataset.queries_iter(), taat)

In [ ]:
query_processing(queries_dataset.queries_iter(), daat)